## TF - IDF(단어 빈도 - 역 문서 빈도, Term Frequency-Inverse Document Frequency)

- **단어의 빈도와 역 문서 빈도(문서의 빈도에 특정 식을 취함)을 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법이다.**


- TF-IDF를 사용하면, 기존의 DTM을 사용하는 것보다 더 많은 정보를 고려하여 문서들을 비교할 수 있다.


- 우선 DTM을 만든 후, TF-IDF 가중치를 부여한다.


- **TF-IDF는 주로 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 쓰일 수 있다.**


- TF-IDF는 TF와 IDF를 곱한 값을 의미하며 문서를 d, 단어를 t, 문서의 총 개수를 n이라고 표현한다.

***

### 1) tf(d,t) : 특정 문서 d에서의 특정 단어 t의 등장 횟수

- DTM이 각 문서에서의 각 단어의 등장 빈도를 나타내는 값이었다. 따라서 TF는 DTM의 각 단어들이 가진 값이라고 볼 수 있다.

### 2) df(t) :  특정 단어 t가 등장한 문서의 수

- 특정 단어가 각 문서, 또는 문서들에서 몇 번 등장했는지는 관심가지지 않으며 오직 특정 단어 t가 등장한 문서의 수에만 관심을 가진다.

예를들어 DTM에서 '바나나'라는 단어가 문서2와 문서3에서 등장했다면 이 경우 바나나의 df는 2다. 하나의 문서에서 바나나라는 단어가 몇 번 등장했다는 건 중요한게 아니다. 아무리 많이 등장했다 하더라도 바나나의 df는 2다.


### 3) idf(d,t) : df(t)에 반비례하는 수

$$idf(d,t) = log({n \over1+df(t)})$$

- IDF는 DF의 역수를 취한 것이다. 왜 log와 분모에 1을 더해주는 식인지는 살펴볼 필요가 있다. log의 밑은 10을 사용한다고 가정할 때 결과는 다음과 같다.


### idf에 log를 사용한 경우


$idf(d,t) = log({n/df(t)})$<br>
**총 문서의 수 : $n = 1,000,000$**

In [1]:
import numpy as np
import pandas as pd
df_t = [1,100,1000,10000,100000,1000000]
result = 0
idf_df = pd.DataFrame()
n = 1000000
for idx,val in enumerate(df_t) :
    result = np.log10(n/val)
    idf_dict = {"단어 t" : "word{}".format(int(idx)), "df(t)" : val, "idf(d,t)" : result}
    idf_df = idf_df.append(idf_dict, ignore_index = True)

In [2]:
idf_df

,단어 t,df(t),"idf(d,t)"
0,word0,1.0,6.0
1,word1,100.0,4.0
2,word2,1000.0,3.0
3,word3,10000.0,2.0
4,word4,100000.0,1.0
5,word5,1000000.0,0.0


### log를 사용하지 않은 경우

$idf(d,t) = n/df(t)$
**총 문서의 수 : $n = 1,000,000$**

In [3]:
df_t = [1,100,1000,10000,100000,1000000]
result = 0
idf_df = pd.DataFrame()
n = 1000000
for idx,val in enumerate(df_t) :
    result = n/val
    idf_dict = {"단어 t" : "word{}".format(int(idx)), "df(t)" : val, "idf(d,t)" : result}
    idf_df = idf_df.append(idf_dict, ignore_index = True)

In [4]:
idf_df

,단어 t,df(t),"idf(d,t)"
0,word0,1.0,1000000.0
1,word1,100.0,10000.0
2,word2,1000.0,1000.0
3,word3,10000.0,100.0
4,word4,100000.0,10.0
5,word5,1000000.0,1.0


log를 사용하지 않았을때, IDF를 DF의 역수로 사용한다면 총 문서의 수 n이 커질 수록, IDF의 값은 기하급수적으로 커지게 된다. 그렇기 때문에 log를 사용한다.

한마디로 **log를 씌우면 자주 쓰이지 않는 단어에 너무 큰 가중치를 부여하는 것을 방지 하게된다.**

### 분모에 1을 더해주는 이유

- 특정 단어가 전체 문서에서 등장하지 않을 경우 분모가 0이 되는 상황을 방지하기 위함

- TF-IDF는 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단하며, 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단한다. TF-IDF값이 낮으면 중요도가 낮은 것이며 TF-IDF 값이 크면 중요도가 큰 것이다.

즉, the나 a와 같이 불용어의 경우에는 모든 문서에 자주 등장하기 마련이기 때문에 자연스럽게 불용어의 TF-IDF값은 다른 단어의 TF-IDF에 비해서 낮아지게 된다.

## 2. 파이썬으로 TF-IDF 직접 구현

DTM을 만들어 TF-IDF를 이해해보자.


In [5]:
import pandas as pd
from math import log

In [6]:
# 4개의 문서 생성, 저장
docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 


vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
vocab

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

#### TF, IDF, TF-IDF 값을 구하는 함수를 구현

In [7]:
N = len(docs) # 총 문서의 수

def tf(t, d) :
    return d.count(t)

def idf(t) :
    df = 0
    for doc in docs :
        df += t in doc
    return log(N/(df + 1))
    
def tfidf(t,d) :
    return tf(t,d)*idf(t)

## TF
우선 **TF**는 앞서 사용한 DTM을 그대로 사용하면, 그것이 각 문서에서의 각 단어의 TF가 된다.

***
~~~python
def tf(t, d) :
    return d.count(t)
~~~
***

#### TF를 구하기(DTM을 데이터프레임에 저장하여 출력)

In [8]:
result = []
for i in range(N) : # 각 문서에 대하여 아래 명령을 수행
    result.append([])
    d = docs[i]
    for j in range(len(vocab)) : # 각 문서별, vocab을 count
        t = vocab[j]
        result[-1].append(tf(t,d))
        
tf_ = pd.DataFrame(result, columns = vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


### IDF


그다음 구해야할 것은 TF와 곱해야할 값인 IDF다. 로그는 밑이 자연 상수 e를 사용하는 자연로그를 사용하여 구한다

IDF 계산을 위해 사용하는 로그의 밑은 TF-IDF를 사용하는 사용자가 임의로 정할 수 있는데, 여기서 로그는 마치 기존의 값에 곱하여 값의 크기를 조절하는 상수의 역할을 한다.

보통 각종 프로그래밍 언어나 패키지에서 지원하는 TF-IDF의 로그는 대부분 자연 로그를 사용한다.

***
~~~python 
def idf(t) :
    df = 0
    for doc in docs :
        df += t in doc
    return log(N/df + 1)

~~~
***
|단어|IDF(역 문서 빈도)|
|---|---|
|**과일이**|**$ln(4/1+1))$ = 0.693147**|
|**길고**|**$ln(4/1+1))$ = 0.693147**|
|**노란**|**$ln(4/1+1))$ = 0.693147**|
|**먹고**|**$ln(4/2+1))$ = 0.287682**|
|**바나나**|**$ln(4/2+1))$ = 0.287682**|
|**사과**|**$ln(4/1+1))$ = 0.693147**|
|**싶은**|**$ln(4/2+1))$ = 0.287682**|
|**저는**|**$ln(4/1+1))$ = 0.693147**|
|**좋아요**|**$ln(4/1+1))$ = 0.693147**|

***
문서의 총 수(n)는 4이기 때문에 ln 안에서 분자는 늘 4로 동일하다.

분모의 경우에는 각 단어가 등장한 문서의 수(DF)를 의미하는데, '과일이'의 경우 4개의 문서 중 문서1, 총 1개의 문서에만 등장했기 때문에 1이라는 값을 가진다. 


#### idf 구하기

In [9]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns = ["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


## TF-IDF

TF는 DTM을 그대로 가져오면 각 문서에서의 각 단어의 TF를 가져오게 되기 때문에, 앞서 사용한 DTM에서 단어 별로 위의 IDF값을 그대로 곱해주면 TF-IDF가 나오게 된다.


In [10]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


TF-IDF를 살펴보면 **문서2에서의 바나나의 TF-IDF 가중치와 문서3에서의 바나나의 TF-IDF 가중치가 다르게 나온다.**

TF는 각각 1과 2로 달랐다는 이유도 있지만, TF-IDF의 관점에서 보자면 TF-IDF는 특정 문서에서 자주 등장하는 단어는 그 문서 내에서 중요한 단어로 판단하기 때문이다

따라서, 문서2에서 바나나를 1번 언급, 문서3에서는 바나나를 2번 언급했기 때문에 문서3에서의 바나나를 더욱 중요한 단어라고 볼 수 있다.

## 사이킷런을 이용한 DTM과 TF-IDF

사이킷런으로도 DTM과 TF-IDF를 만들 수 있다. BoW를 만들때 사용한 **CouterVectorizer**사용하여 DTM을 만든다.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ', ]

In [12]:
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray()) # 각 코퍼스로부터 각 단어의 빈도 수를 기록
print(vector.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여줌.

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


#### 첫 번째 열은 0의 인덱스를 가진 do(do는 3번째 문서에만 등장했으므로 3번째 행에서만 1의 값을 가진다)


#### 두번째 열의 경우는 1의 인덱스를 가진 know다. know는 1번째 문서에만 등장했으므로 1번째 행에서만 1의 값을 가진다.

### 사이킷런을 사용하여 TF-IDF 계산

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]

tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


사이킷런의 TF-IDF는 위에서 사용한 보편적인 TF-IDF 식에서 좀 더 조정된 다른 식을 사용한다

그러나 여전히 TF-IDF가 가진 의도를 그대로 갖고 있으므로 사이킷런의 TF-IDF를 사용해도 좋다.